<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, Jonathan Morgan, Ridhima Sodhi, Ursula Kaczmarek. 

_source to be updated when notebook added to GitHub_

# Data Visualization in Python
---

## Table of Contents

- [Introduction](#Introduction)

- [Python Setup](#Python-Setup)

- [Load the Data](#Load-the-Data)

- [Visual Data Exploration with Matplotlib](#Visual-data-exploration-with-matplotlib)
    - [A Note on Data Sourcing and Labeling](#A-Note-on-Data-Sourcing-and-Labeling)
    - [Mapping with matplotlib](#Mapping-with-matplotlib)
    
    
- [Introducing seaborn](#Introducing-seaborn)


## Introduction
- Back to [Table of Contents](#Table-of-Contents)

In this module, you will learn to quickly and flexibly make a wide series of visualizations for exploratory data analysis. This module contains a practical introduction to data visualization in Python and covers important rules that any data visualizer should follow.

### Learning Objectives

* Become familiar with a core base of data visualization tools in Python - specifically `matplotlib` and `seaborn`

* Begin exploring what visualizations are going to best reveal various types of patterns in your data

* Learn more about our primary datasets data with exploratory analyses and visualizations

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# data manipulation in Python
import pandas as pd


# numeric operations
import numpy as np

# visualization packages
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import seaborn as sn

# geography package
import geopandas as gpd
import shapely

# database connection
import psycopg2
import sqlalchemy

# see how long queries and cell content take to run
import time

# so images get plotted in the notebook
%matplotlib inline

## Load the Data
- Back to [Table of Contents](#Table-of-Contents)

Here, we build on the TANF experience explored in the [dataset exploration notebook](01_2_Dataset_Exploration.ipynb) through visualization. We also visualize labor market outcomes of TANF members. For analytical purposes, we will focus on young adult members (we define members as the individuals who qualify for and receive cash benefits) who ended an Indiana or Illinois TANF spell in 2014 Q4 and who were also aged 18-30 at that time. 

Our guiding questions are as follows:
- What are the different ways of visualizing the different TANF experiences of the cohort?
- What are the different ways of visualizing the labor market outcomes of this cohort?

In [ ]:
# set up sqlalchemy engine
host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = sqlalchemy.create_engine(connection_string)

In [ ]:
# let's start by pulling data on our 2014 Q4 ending spell cohort

start_time = time.time()

query = """
DROP TABLE IF EXISTS cohort_2014;

CREATE TEMP TABLE cohort_2014 AS
-- Illinois cohort members
SELECT DISTINCT ON (m.ssn_hash) ssn_hash AS member_ssn, 17 AS state, '2014-10-1'::date as end_yr_q,  
DATE_PART('year', AGE(i.end_date, m.birth_date))::INT AS member_age, 

    -- calculate length of spell that is ending this quarter 
    -- epochs are measured in seconds, so we want to divide by the number of seconds in a month 
    EXTRACT(EPOCH FROM AGE(i.end_date, i.start_date))/(3600.*672) as spell_len_months

FROM il_dhs.indcase_spells i, il_dhs.member_relation r, il_dhs.member m
WHERE i.recptno = r.recptno AND i.ch_dpa_caseid = r.ch_dpa_caseid 
AND i.recptno = m.recptno AND i.ch_dpa_caseid = m.ch_dpa_caseid
AND i.end_date BETWEEN '2014-10-01'::DATE AND '2014-12-31'::DATE 
AND i.benefit_type = 'tanf46'
AND reltogte = 82
AND DATE_PART('year', AGE(i.end_date, m.birth_date))::INT >= 18 
    AND DATE_PART('year', AGE(i.end_date, m.birth_date))::INT <= 30

-- Indiana cohort members
UNION ALL 
SELECT DISTINCT ON (ssn) ssn AS member_ssn, 18 AS state, '2014-10-1'::date as end_yr_q,
DATE_PART('year', AGE(tanf_end_date::DATE, dob::DATE))::INT AS member_age,

-- calculate length of spell that is ending this quarter
    EXTRACT(EPOCH FROM AGE(tanf_end_date, tanf_start_date))/(3600.*672) as spell_len_months

FROM in_fssa.person_month 
WHERE tanf_end_date::DATE BETWEEN '2014-10-01'::DATE AND '2014-12-31'::DATE
AND tanf = 1
AND affil = '1'
AND DATE_PART('year', AGE(tanf_end_date::DATE, dob::DATE))::INT >= 18 AND
       DATE_PART('year', AGE(tanf_end_date::DATE, dob::DATE))::INT <= 30;

COMMIT;
"""
conn.execute(query)

# report how long creating this table took
print('query ran in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
# eyeball the tanf data
df_tanf = pd.read_sql('SELECT * FROM cohort_2014', conn)
df_tanf.head()

In [ ]:
# how many unique members are there in the cohort?
df_tanf['member_ssn'].nunique()

In addition to exploring the TANF experience, we are interested in looking at the characteristics of jobs generating wages for our cohort in the year after their spells end. We will look at two aspects of labor market outcomes:
- the size of firms employing members of our cohort (as measured by number of employees)
- the quarter over quarter percent change in average wages within the industries in which our cohort members find employment

In [ ]:
# pull our cohort's multi-state jobs data
query = """
DROP TABLE IF EXISTS cohort_jobs;

CREATE TEMP TABLE cohort_jobs AS
SELECT w.*, SUBSTRING(e.naics, 1, 2) as naics
FROM ada_tdc_2019.q42014_cohort_wage w, ada_tdc_2019.tanf_employers e
WHERE w.uiacct = e.uiacct
AND w.state = e.state
ORDER BY job_yr_q;

COMMIT;
"""
conn.execute(query)
print('query complete in {:.2f} seconds'.format(time.time()-start_time))

To calculate average wages within industries, we must group by naics codes and year/quarter.

In [ ]:
df_jobs = pd.read_sql('select * from cohort_jobs', conn)
df_jobs = df_jobs.groupby(['naics', 'job_yr_q'])['wages'].mean().reset_index()
df_jobs.head()

In [ ]:
# pull our employer firms' geographic characteristics
# Indiana does not have geographic data so we will focus on Illinois
sql = """
DROP TABLE IF EXISTS il_cohort_firms; 

CREATE TEMP TABLE il_cohort_firms AS
SELECT county, empr_no, SUBSTRING(naics, 1, 2) as naics,
GREATEST(empl_month1::INT, empl_month2::INT, empl_month3::INT) AS firm_size,
CAST(longitude as DOUBLE PRECISION) AS longitude,
CAST(latitude as DOUBLE PRECISION) AS latitude
FROM il_des_kcmo.il_qcew_employers 
WHERE empr_no IN (SELECT DISTINCT ON (uiacct) uiacct FROM cohort_jobs)
GROUP BY county, empr_no, naics, longitude, latitude, empl_month1, empl_month2, empl_month3;
   
COMMIT;
"""
conn.execute(sql)
print('query complete in {:.2f} seconds'.format(time.time()-start_time))

## Visual data exploration with matplotlib
- Back to [Table of Contents](#Table-of-Contents)

Under the hood, `Pandas` uses `matplotlib` to produce visualizations. `matplotlib` is the most commonly used base visualization package and allows for a high degree of customization.

In [ ]:
# simple tanf spell length distributions: histograms
df_tanf.spell_len_months.plot(kind='hist', bins=50, alpha=0.6, figsize=(15,5))
plt.ylabel('count')
plt.xlabel('spell length (months)')
plt.title('distribution of cohort spell lengths')
plt.annotate('Sources: Illinois Department of Human Services, Indiana Family and Social Services Administration ', 
             xy=(0.75,-0.15), xycoords="axes fraction");

Let's visualize spell length for the younger and older members of our cohort.

In [ ]:
# let's label our cohort members as younger or older
df_tanf['age_bin'] = pd.cut(df_tanf['member_age'], bins=[18, 25, 30], labels=['younger', 'older'])

In [ ]:
df_tanf.loc[df_tanf['age_bin']=='younger'].spell_len_months.plot(kind='hist', bins=50, alpha=0.6, figsize=(15,5))
df_tanf.loc[df_tanf['age_bin']=='older'].spell_len_months.plot(kind='hist', bins=50, alpha=0.6, figsize=(15,5))
plt.ylabel('count')
plt.xlabel('spell length (months)')
plt.title('distribution of cohort spell lengths by age group')
plt.legend(['younger (<= 25)', 'older (> 25)'], frameon=False)
plt.annotate('Sources: Illinois Department of Human Services, Indiana Family and Social Services Administration', 
             xy=(0.75,-0.15), xycoords="axes fraction");

Our spell length distributions are heavily skewed right. What do they look like if we remove outliers?

Let's break down the length of our cohort's spells by state and visualize the distributions using another effective chart type for presenting parameters of distributions: boxplots. 

> Note: Keep in mind that because you cannot export individual observations from the ADRF, you should really only use boxplots here to help guide your data exploration.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5)) # this makes it easier to shape a long x-axis
df_tanf_il = df_tanf.loc[df_tanf['state']==17]
df_tanf_in = df_tanf.loc[df_tanf['state']==18]
ax.boxplot([df_tanf_il.spell_len_months, df_tanf_in.spell_len_months], vert=False)
ax.set(xlabel='spell length', yticklabels=['illinois', 'indiana'],\
       title='distribution of cohort member spell lengths by state')
plt.annotate('Sources: Illinois Department of Human Services, Indiana Family and Social Services Administration', 
             xy=(0.75,-0.15), xycoords="axes fraction");

In [ ]:
# with 99th percentile removed
fig, ax = plt.subplots(figsize=(15, 5)) # this makes it easier to shape a long x-axis
ax.boxplot([df_tanf_il[df_tanf_il.spell_len_months < df_tanf_il.spell_len_months.quantile(0.99)].spell_len_months,\
            df_tanf_in[df_tanf_in.spell_len_months < df_tanf_in.spell_len_months.quantile(0.99)].spell_len_months], vert=False)
ax.set(xlabel='spell length (months)', yticklabels=['illinois', 'indiana'],\
       title='distribution of cohort member spell lengths by state (< 99th percentile)')
plt.annotate('Sources: Illinois Department of Human Services, Indiana Family and Social Services Administration', 
             xy=(0.75,-0.15), xycoords="axes fraction");

### A Note on Data Sourcing and Labeling
- Back to [Table of Contents](#Table-of-Contents)

Data sourcing and labeling axes are critical to effectively conveying information through visualization. Although here we are simply referencing the agencies that created the data, it is ideal to provide as direct a path as possible for the viewer to find the sourced data. When this is not possible (e.g. the data is sequestered), directing the viewer to documentation or methodology for the data is a good alternative. Regardless, providing clear labels and sourcing for the underlying data is an **absolute requirement** of any respectable visualization and further builds trust and enables reproducibility.

### Mapping with matplotlib
- Back to [Table of Contents](#Table-of-Contents)

Let's learn a bit more about the geographic distribution of firms employing members of the cohort.

It contains the firms' county locations, point geographies (latitude, longitude coordinates) of the firm locations and the year of their incorporation. We can read these into a regular `pandas` dataframe and then convert to a `geopandas` geodataframe. A geodataframe has a geometry column of formatted geographies that tells `matplotlib` where the points, polygons, lines, etc. belong on the map.  

Again, because `matplotlib` requires a geometric object to know where to put a point on a map, not just the floats that make up latitude and longitude values, we must add that object with `geopandas` and make a geodataframe.

In addition to the geometric object, we must specify the kind of map projection we want. There are numerous coordinate  systems to choose from; the [epsg](http://www.epsg.org) 4269 projection is present in another geodataframe we will use a bit later.

In [ ]:
# first read in our data (where we have coordinate values) as a regular pandas dataframe
firms = pd.read_sql('select * from il_cohort_firms where firm_size > 0 and latitude is not null', conn).drop_duplicates()

# create our geometric object using the `shapely` package
geometry = [shapely.geometry.Point(xy) for xy in zip(firms.longitude, firms.latitude)]


# convert our dataframe into a geodataframe: be sure to specify both projection and geometry
# we can also drop the latitude & longitude columns as they're now redundant
geo_jobs = gpd.GeoDataFrame(firms, geometry = geometry).drop(columns = ['latitude', 'longitude'])
geo_jobs.crs = {'init':'epsg:4269'}
geo_jobs.head()

We have quite a bit of range in our firm sizes, so let's group them into intervals, just like we did above for ages.

In [ ]:
geo_jobs['size_bin'] = pd.cut(geo_jobs['firm_size'], bins=[1, 500, 1000, geo_jobs['firm_size'].max()],\
                              labels=[1,2,3])
        
geo_jobs.head()

We need one last thing: the contours of Illinois to give context to our points. We can retrieve the polygons that make up counties within Illinois from the `public.tl_2016_us_county` table. We can filter to Illinois by specifying its state FIPS code using a `WHERE` clause in SQL.

In [ ]:
query = """
SELECT statefp, countyfp, geom FROM public.tl_2016_us_county WHERE statefp = '17'
"""
il_polygons = gpd.GeoDataFrame.from_postgis(query, conn, geom_col='geom')
il_polygons.crs = geo_jobs.crs

In [ ]:
# now lets plot
# we don't really need to see the coordinate values on the axes, so we'll suppress them
# we need separate plot() calls for the polygons and the points, but they both need to reference the same axes

# create a custom color map for our bin sizes
reds = plt.get_cmap('Reds') # use a built-in color palette
bins = list(set(geo_jobs['size_bin']))
cnorm = colors.Normalize(vmin=0, vmax=len(bins))
scalarmap = cmx.ScalarMappable(norm=cnorm, cmap=reds)

# color by the firm size group
groups = geo_jobs.groupby('size_bin')
fig,ax = plt.subplots(figsize=(10, 10))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
il_polygons.plot(ax=ax, alpha=0.2, color='lightgray', edgecolor='cadetblue')
for name, group in groups:
    group.plot(ax=ax, marker='.', label=name, color=scalarmap.to_rgba(name))
plt.legend(['1-500', '500-1000', '1000+'], frameon=False)
plt.title('Locations and sizes of Illinois Firms Employing the TANF Cohort')
plt.annotate('Sources: Illinois Department of Human Services, Indiana Family and Social Services Administration', 
             xy=(0.75,-0.05), xycoords="axes fraction");

We see a lot of occlusion with plotted points, especially in Chicago and other population centers. Let's see if mapping average firm age by county as a choropleth is easier to interpret. We have firm age and county labels in our `geo_jobs` geodataframe, and the county geometries in our `il_polygons` geodataframe. Here we demonstrate how to use the `sjoin()` function to perform a spatial join based on spatial relationships.  

In [ ]:
# we use the 'how' argument to keep all county polygons, even if there aren't firms in it, to keep the map whole
# we use the 'op' argument to tell geopandas that our points reside within our county polygons
geo_jobs = geo_jobs[['county', 'firm_size', 'geometry']]
joined = gpd.sjoin(il_polygons, geo_jobs, how='left')[['geom', 'countyfp', 'firm_size']]
joined.head()

To calculate the mean firm size by county, we have two options:
- `geopandas` `dissolve()` function, which aggregates an attribute value and dissolves all the geometries within a group into a single geometric feature (which has a long runtime)
- group by an attribue and `merge()` to a geodataframe

We will demonstrate the `merge()` process here.

In [ ]:
# calculate the average firm size using a regular dataframe
grouped = firms.groupby('county')['firm_size'].mean().reset_index()
grouped.columns = ['countyfp', 'avg_firm_size']
grouped.head()

In [ ]:
# merge to our polygons
merged = il_polygons.merge(grouped, on='countyfp')
merged.head()

In [ ]:
# let's plot a choropleth of average TANF employer firm size by county in Illinois
fig,ax = plt.subplots(figsize=(10, 10))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
merged.plot(column='avg_firm_size', ax=ax, cmap='Reds', legend=True)
plt.title('Average size of Illinois Firms Employing the TANF Cohort')
plt.annotate('Source: Illinois Department of Employment Security', 
             xy=(0.75,-0.05), xycoords="axes fraction");

## Introducing seaborn
- Back to [Table of Contents](#Table-of-Contents)

`seaborn` is a popular visualization package built on top of `matplotlib` which makes some more complicated graphs easier to make.`seaborn`'s heatmap is an informative way to view a continuous variable. Here we will conclude this notebook by visualizing two categorical variables, industry and year/quarter, and one continuous variable, the percent change in average quarterly wages within industries in which our cohort worked in a heatmap.

In [ ]:
df_jobs['pct_change'] = df_jobs.groupby('naics')['wages'].transform(lambda x: x.pct_change())
df_jobs.dropna(inplace=True)
df_jobs.head(15)

In [ ]:
# now let's plot a heatmap
plt.figure(figsize=(10, 20))
sn.set_style('ticks')
sn.heatmap(df_jobs.pivot('naics', 'job_yr_q', 'pct_change'), cmap='coolwarm')\
.set_title('percentage change in wages within cohort-employing industries')
plt.annotate('Sources: Illinois Department of Human Services, Indiana Family and Social Services Administration', 
             xy=(0.75,-0.05), xycoords="axes fraction");